In [1]:
""" Performing a large-scale experiment on cifar10 """

import keras
import logging
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
import keras.regularizers as regularizers
from keras.models import load_model

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="swish_first_exp_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [3]:
def create(act):
    baseMapNum = 32
    weight_decay = 1e-4

    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    return model

In [4]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [7]:
def schedule(x):
    if x < 75: 
        return 0.001
    elif x < 100:
        return 0.0005
    elif x < 125:
        return 0.0003
    elif x<150:
        return 0.00015
    elif x<175:
        return 0.000075
    elif x<200: 
        return 0.000035
    elif x<225:
        return 0.000017
    else:
        return 0.000012

In [6]:
# Create model and get ready - reproducible cell
act, act_name = "relu", "relu"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=125
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('relu_at_2.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 142s 364ms/step - loss: 0.5766 - acc: 0.8513 - val_loss: 0.4702 - val_acc: 0.8843
Epoch 93/125
390/390 [==============================] - 142s 364ms/step - loss: 0.5672 - acc: 0.8561 - val_loss: 0.4829 - val_acc: 0.8812
Epoch 94/125
390/390 [==============================] - 142s 363ms/step - loss: 0.5708 - acc: 0.8523 - val_loss: 0.4743 - val_acc: 0.8816
Epoch 95/125
390/390 [==============================] - 142s 364ms/step - loss: 0.5769 - acc: 0.8535 - val_loss: 0.4854 - val_acc: 0.8795
Epoch 96/125
390/390 [==============================] - 142s 364ms/step - loss: 0.5685 - acc: 0.8564 - val_loss: 0.4765 - val_acc: 0.8840
Epoch 97/125
390/390 [==============================] - 142s 364ms/step - loss: 0.5668 - acc: 0.8556 - val_loss: 0.4792 - val_acc: 0.8811
Epoch 98/125
390/390 [==============================] - 142s 363ms/step - loss: 0.5623 - acc: 0.8547 - val_loss: 0.4645 - val_acc: 0.8857
Epoch 99/125
390/390 [=========================

In [8]:
# Fit the model
epochs = 150
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1], initial_epoch=125)
# Save the model
model.save('relu_at_2.h5')
# Display logs
print(his.history)

Epoch 126/150
390/390 [==============================] - 144s 370ms/step - loss: 0.4875 - acc: 0.8745 - val_loss: 0.4247 - val_acc: 0.8959
Epoch 127/150
390/390 [==============================] - 143s 366ms/step - loss: 0.4853 - acc: 0.8769 - val_loss: 0.4142 - val_acc: 0.8995
Epoch 128/150
390/390 [==============================] - 143s 366ms/step - loss: 0.4909 - acc: 0.8755 - val_loss: 0.4278 - val_acc: 0.8981
Epoch 129/150
390/390 [==============================] - 143s 366ms/step - loss: 0.4765 - acc: 0.8781 - val_loss: 0.4252 - val_acc: 0.8962
Epoch 130/150
390/390 [==============================] - 143s 366ms/step - loss: 0.4871 - acc: 0.8743 - val_loss: 0.4140 - val_acc: 0.8983
Epoch 131/150
390/390 [==============================] - 142s 364ms/step - loss: 0.4754 - acc: 0.8785 - val_loss: 0.4117 - val_acc: 0.9006
Epoch 132/150
390/390 [==============================] - 142s 364ms/step - loss: 0.4788 - acc: 0.8788 - val_loss: 0.4254 - val_acc: 0.8969
Epoch 133/150
390/390 [====

In [9]:
model = load_model('relu_at_2.h5')
# Fit the model
batch_size = 128
epochs = 250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
#Compile the model again
opt_rms = keras.optimizers.rmsprop(lr=0.000075,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1], initial_epoch=150)
# Save the model
model.save('relu_at_2.h5')
# Display logs
print(his.history)

Epoch 151/250
390/390 [==============================] - 148s 379ms/step - loss: 0.4485 - acc: 0.8838 - val_loss: 0.4056 - val_acc: 0.9003
Epoch 152/250
390/390 [==============================] - 143s 367ms/step - loss: 0.4527 - acc: 0.8834 - val_loss: 0.4075 - val_acc: 0.8992
Epoch 153/250
390/390 [==============================] - 143s 367ms/step - loss: 0.4425 - acc: 0.8855 - val_loss: 0.4035 - val_acc: 0.9007
Epoch 154/250
390/390 [==============================] - 143s 367ms/step - loss: 0.4423 - acc: 0.8861 - val_loss: 0.4013 - val_acc: 0.9009
Epoch 155/250
390/390 [==============================] - 142s 365ms/step - loss: 0.4474 - acc: 0.8845 - val_loss: 0.4014 - val_acc: 0.9007
Epoch 156/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4438 - acc: 0.8841 - val_loss: 0.4067 - val_acc: 0.8999
Epoch 157/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4473 - acc: 0.8848 - val_loss: 0.4016 - val_acc: 0.9004
Epoch 158/250
390/390 [====

Epoch 210/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4161 - acc: 0.8918 - val_loss: 0.3911 - val_acc: 0.9026
Epoch 211/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4184 - acc: 0.8928 - val_loss: 0.3927 - val_acc: 0.9019
Epoch 212/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4162 - acc: 0.8927 - val_loss: 0.3938 - val_acc: 0.9017
Epoch 213/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4182 - acc: 0.8925 - val_loss: 0.3945 - val_acc: 0.9012
Epoch 214/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4164 - acc: 0.8923 - val_loss: 0.3919 - val_acc: 0.9011
Epoch 215/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4135 - acc: 0.8941 - val_loss: 0.3926 - val_acc: 0.9018
Epoch 216/250
390/390 [==============================] - 142s 364ms/step - loss: 0.4133 - acc: 0.8925 - val_loss: 0.3919 - val_acc: 0.9022
Epoch 217/250
390/390 [====

In [10]:
# Create model and get ready - reproducible cell
act, act_name = "relu", "relu"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=200
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('relu_at_3.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 142s 364ms/step - loss: 0.7532 - acc: 0.7979 - val_loss: 0.6017 - val_acc: 0.8417
Epoch 34/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7705 - acc: 0.7946 - val_loss: 0.6732 - val_acc: 0.8328
Epoch 35/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7517 - acc: 0.7981 - val_loss: 0.6077 - val_acc: 0.8391
Epoch 36/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7458 - acc: 0.7984 - val_loss: 0.5767 - val_acc: 0.8505
Epoch 37/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7471 - acc: 0.7993 - val_loss: 0.6064 - val_acc: 0.8429
Epoch 38/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7408 - acc: 0.8023 - val_loss: 0.6010 - val_acc: 0.8422
Epoch 39/200
390/390 [==============================] - 142s 364ms/step - loss: 0.7437 - acc: 0.8013 - val_loss: 0.5859 - val_acc: 0.8505
Epoch 40/200
390/390 [=========================

390/390 [==============================] - 142s 364ms/step - loss: 0.5442 - acc: 0.8552 - val_loss: 0.4543 - val_acc: 0.8901
Epoch 93/200
390/390 [==============================] - 142s 364ms/step - loss: 0.5357 - acc: 0.8575 - val_loss: 0.4981 - val_acc: 0.8768
Epoch 94/200
390/390 [==============================] - 142s 364ms/step - loss: 0.5400 - acc: 0.8552 - val_loss: 0.4722 - val_acc: 0.8852
Epoch 95/200
390/390 [==============================] - 142s 364ms/step - loss: 0.5358 - acc: 0.8576 - val_loss: 0.4596 - val_acc: 0.8872
Epoch 96/200
390/390 [==============================] - 142s 365ms/step - loss: 0.5374 - acc: 0.8553 - val_loss: 0.4757 - val_acc: 0.8844
Epoch 97/200
390/390 [==============================] - 142s 364ms/step - loss: 0.5378 - acc: 0.8568 - val_loss: 0.4681 - val_acc: 0.8855
Epoch 98/200
390/390 [==============================] - 142s 364ms/step - loss: 0.5306 - acc: 0.8582 - val_loss: 0.4627 - val_acc: 0.8852
Epoch 99/200
390/390 [=========================

390/390 [==============================] - 142s 364ms/step - loss: 0.4195 - acc: 0.8864 - val_loss: 0.3925 - val_acc: 0.9012
Epoch 152/200
390/390 [==============================] - 142s 364ms/step - loss: 0.4110 - acc: 0.8892 - val_loss: 0.3940 - val_acc: 0.9026
Epoch 153/200
390/390 [==============================] - 142s 364ms/step - loss: 0.4105 - acc: 0.8887 - val_loss: 0.3894 - val_acc: 0.9039
Epoch 154/200
390/390 [==============================] - 142s 364ms/step - loss: 0.4097 - acc: 0.8892 - val_loss: 0.3901 - val_acc: 0.9009
Epoch 155/200
390/390 [==============================] - 142s 365ms/step - loss: 0.4117 - acc: 0.8895 - val_loss: 0.3905 - val_acc: 0.9030
Epoch 156/200
390/390 [==============================] - 142s 364ms/step - loss: 0.4097 - acc: 0.8905 - val_loss: 0.3883 - val_acc: 0.9034
Epoch 157/200
390/390 [==============================] - 142s 364ms/step - loss: 0.4152 - acc: 0.8880 - val_loss: 0.3860 - val_acc: 0.9038
Epoch 158/200
390/390 [==================

In [12]:
epochs = 250
model = load_model('relu_at_3.h5')
opt_rms = keras.optimizers.rmsprop(lr=0.000017,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1], initial_epoch=200)
# Save the model
model.save('relu_at_3.h5')
# Display logs
print(his.history)

Epoch 201/250
390/390 [==============================] - 144s 370ms/step - loss: 0.3826 - acc: 0.8967 - val_loss: 0.3779 - val_acc: 0.9063
Epoch 202/250
390/390 [==============================] - 143s 366ms/step - loss: 0.3823 - acc: 0.8968 - val_loss: 0.3773 - val_acc: 0.9072
Epoch 203/250
390/390 [==============================] - 143s 367ms/step - loss: 0.3856 - acc: 0.8974 - val_loss: 0.3766 - val_acc: 0.9069
Epoch 204/250
390/390 [==============================] - 143s 367ms/step - loss: 0.3803 - acc: 0.8979 - val_loss: 0.3789 - val_acc: 0.9064
Epoch 205/250
390/390 [==============================] - 143s 368ms/step - loss: 0.3843 - acc: 0.8954 - val_loss: 0.3786 - val_acc: 0.9070
Epoch 206/250
390/390 [==============================] - 143s 367ms/step - loss: 0.3842 - acc: 0.8968 - val_loss: 0.3771 - val_acc: 0.9068
Epoch 207/250
390/390 [==============================] - 143s 367ms/step - loss: 0.3845 - acc: 0.8976 - val_loss: 0.3755 - val_acc: 0.9065
Epoch 208/250
390/390 [====

In [13]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [14]:
import keras.regularizers as regularizers
ensembler = 0
for i in range(3):
    if i == 0:
        model = create("relu")
        model.load_weights('relu_at_'+str(i+1)+'.h5')
        model.save('relu_at_'+str(i+1)+'.h5')
    else:
        model = load_model('relu_at_'+str(i+1)+'.h5')
    ensembler += model.predict_proba(x_test)
    
print(accuracy(ensembler, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [9]:
# Create model and get ready - reproducible cell
act, act_name = "relu", "relu"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - 
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('relu_at_2.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 174s 447ms/step - loss: 0.5423 - acc: 0.8576 - val_loss: 0.4586 - val_acc: 0.8902
Epoch 93/200
390/390 [==============================] - 174s 446ms/step - loss: 0.5474 - acc: 0.8566 - val_loss: 0.4638 - val_acc: 0.8858
Epoch 94/200
390/390 [==============================] - 174s 446ms/step - loss: 0.5499 - acc: 0.8535 - val_loss: 0.4542 - val_acc: 0.8877
Epoch 95/200
390/390 [==============================] - 174s 446ms/step - loss: 0.5362 - acc: 0.8586 - val_loss: 0.4561 - val_acc: 0.8869
Epoch 96/200
390/390 [==============================] - 174s 447ms/step - loss: 0.5433 - acc: 0.8543 - val_loss: 0.4426 - val_acc: 0.8915
Epoch 97/200
390/390 [==============================] - 174s 446ms/step - loss: 0.5360 - acc: 0.8568 - val_loss: 0.4597 - val_acc: 0.8866
Epoch 98/200
390/390 [==============================] - 174s 446ms/step - loss: 0.5389 - acc: 0.8566 - val_loss: 0.4537 - val_acc: 0.8865
Epoch 99/200
390/390 [=========================

390/390 [==============================] - 174s 446ms/step - loss: 0.4115 - acc: 0.8871 - val_loss: 0.3882 - val_acc: 0.9029
Epoch 152/200
390/390 [==============================] - 174s 447ms/step - loss: 0.4149 - acc: 0.8877 - val_loss: 0.3919 - val_acc: 0.9037
Epoch 153/200
390/390 [==============================] - 174s 447ms/step - loss: 0.4113 - acc: 0.8888 - val_loss: 0.3920 - val_acc: 0.9013
Epoch 154/200
390/390 [==============================] - 174s 447ms/step - loss: 0.4110 - acc: 0.8882 - val_loss: 0.3838 - val_acc: 0.9035
Epoch 155/200
390/390 [==============================] - 176s 452ms/step - loss: 0.4095 - acc: 0.8899 - val_loss: 0.3948 - val_acc: 0.9024
Epoch 156/200
390/390 [==============================] - 176s 452ms/step - loss: 0.4084 - acc: 0.8893 - val_loss: 0.3909 - val_acc: 0.9042
Epoch 157/200
390/390 [==============================] - 176s 452ms/step - loss: 0.4045 - acc: 0.8903 - val_loss: 0.3851 - val_acc: 0.9046
Epoch 158/200
390/390 [==================

In [ ]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [ ]:
ensembler = 0
for i in range(3):
    model = load_model('relu_at_+'str(i)'+.h5')
    ensembler += model.predict_proba(x_test)
    
print(accuracy(ensembler, y_test))